https://jira.x5.ru/browse/CVMUPG-911_ecomilk

In [1]:
name = 'CVMUPG-911_ecomilk'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='cvm5-rnd', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/13 13:55:49 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Проверка на доступность плю в магазинах

GEO1 - MSK, GEO2 - MSK, SZFO

In [7]:
macroregion_dk1 = ['MRDCT']
macroregion_dk2 = ['MRDCT', 'MRDND']

In [8]:
plu_codes = [4167291,4138521]

In [9]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops1 = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject1))
            .filter(F.col('macroregion_dk').isin(macroregion_dk1))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )
shops2 = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject1))
            .filter(F.col('macroregion_dk').isin(macroregion_dk2))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants1 = (shops1
                .join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

plants2 = (shops2
                .join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )


print(len(plants1))
print(len(plants2))

[Stage 9:======================================================>(297 + 3) / 300]

3301
4992


### Выбираем гостей нужного юзкейса

In [10]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()
start_date = datetime.date(2021, 10, 10)
end_date = datetime.date(2021, 12, 10)

In [11]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [12]:
customers_usecase.count()

15933079

### Проверяем на доступность отобранных гостей в определенную дату

In [13]:
check_date = '2022-01-05'

In [14]:
guests = remove_global_control_group(spark, customers_usecase, check_date)

 Фильтр Глобальная контрольная группа - валиден.


[Stage 20:=====================================================>(298 + 2) / 300]

	 Отсёк 1 236 971 записей.


In [15]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [16]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [17]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:02


In [18]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [19]:
# seg_sms.write.parquet('temp911_1', mode='overwrite')
seg_sms = spark.read.parquet('temp911_1')

In [20]:
seg_sms.count()

8457264

# seg1

### Собираем чеки этих гостей

##### Отбираем PLU определенной категории (сыры мягкие), но не бренд экомилк

In [21]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )


In [22]:
checks_headers_tc5_1 = checks_headers.filter(F.col('store_id').isin(plants1)) #оставили чеки только с нужными магазинами

In [23]:
checks_headers_tc5_2 = checks_headers.filter(F.col('store_id').isin(plants2)) #оставили чеки только с нужными магазинами

In [24]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [25]:
checks_headers_tc5_1 = checks_headers_tc5_1.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [26]:
checks_headers_tc5_2 = checks_headers_tc5_2.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [27]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


##### Отбираем PLU определенной категории (сыры мягкие), но не бренд экомилк

In [28]:
# plu_hierarchy_lvl_4_dk = ['FR0606001']
syntethic_category_id = [102]
# plu_brand_code = ['5256']
plu_brand_code_not_in = ['L312']
# plu_not_in = [4138521]
# plu_id = [3466978]

In [29]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
#                  .filter(F.col('plu_id').isin(plu_id))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [30]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [31]:
checks_tc5_1 = checks_items.join(checks_headers_tc5_1, on='rtl_txn_id', how='inner')

In [32]:
checks_tc5_2 = checks_items.join(checks_headers_tc5_2, on='rtl_txn_id', how='inner')

In [33]:
seg1 = (checks_tc5_1
           .withColumn('geo', F.lit('geo1'))
           .select('customer_rk', 'geo')
           .distinct()
      )

In [34]:
seg2 = (checks_tc5_2
           .withColumn('geo', F.lit('geo2'))
           .select('customer_rk', 'geo')
           .distinct()
      )

In [35]:
seg = seg1.union(seg2)

In [36]:
seg.write.parquet('temp911_2', mode='overwrite')
seg= spark.read.parquet('temp911_2')

In [53]:
seg.count()

2383972

### Собираем чеки этих гостей

In [37]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )


In [38]:
checks_headers_tc5_1 = checks_headers.filter(F.col('store_id').isin(plants1)) #оставили чеки только с нужными магазинами

In [39]:
checks_headers_tc5_2 = checks_headers.filter(F.col('store_id').isin(plants2)) #оставили чеки только с нужными магазинами

In [40]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [41]:
checks_headers_tc5_1 = checks_headers_tc5_1.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [42]:
checks_headers_tc5_2 = checks_headers_tc5_2.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [43]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [44]:
# plu_hierarchy_lvl_4_dk = ['FR0606001']
syntethic_category_id = [102]
plu_brand_code = ['L312']
# plu_brand_code_not_in = ['L312']
# plu_not_in = [4138521]
# plu_id = [3466978]

In [45]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
#                  .filter(F.col('plu_id').isin(plu_id))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [46]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [47]:
checks_tc5_1 = checks_items.join(checks_headers_tc5_1, on='rtl_txn_id', how='inner')

In [48]:
checks_tc5_2 = checks_items.join(checks_headers_tc5_2, on='rtl_txn_id', how='inner')

In [49]:
seg3 = (checks_tc5_1
           .withColumn('geo', F.lit('geo1'))
           .select('customer_rk', 'geo')
           .distinct()
      )

In [50]:
seg4 = (checks_tc5_2
           .withColumn('geo', F.lit('geo2'))
           .select('customer_rk', 'geo')
           .distinct()
      )

In [51]:
seg_2 = seg3.union(seg4)

In [52]:
seg_2.write.parquet('temp911_3', mode='overwrite')
seg_2 = spark.read.parquet('temp911_3')

In [54]:
seg_2.count()

33886

In [55]:
seg1_pd = seg.toPandas()

In [56]:
seg2_pd = seg_2.toPandas()

In [60]:
seg_1_geo_1 = seg1_pd[seg1_pd['geo'] == 'geo1'].reset_index(drop=True)
seg_1_geo_2 = seg1_pd[seg1_pd['geo'] == 'geo2'].reset_index(drop=True)

In [61]:
seg_2_geo_1 = seg2_pd[seg2_pd['geo'] == 'geo1'].reset_index(drop=True)
seg_2_geo_2 = seg2_pd[seg2_pd['geo'] == 'geo2'].reset_index(drop=True)

In [63]:
print(len(seg_1_geo_1))
print(len(seg_1_geo_2))
print(len(seg_2_geo_1))
print(len(seg_2_geo_2))

954296
1429676
14947
18939


In [ ]:
seg_pd = seg_pd.astype('str')

In [ ]:
cross, upgrade = get_usecases(seg_pd, usecases='short')

In [ ]:
seg_no= 'seg1'

In [ ]:
usecase = ['cross', 'upgrade']

In [ ]:
dt = datetime.datetime.now()

In [ ]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [ ]:
print('seg_no: {}, cross: {}, upgrade: {}'.format(seg_no, len(cross), len(upgrade)))